In [ ]:
#!/usr/bin/env python3

# Fetch the HTML for the API, using webdriver and firefox

from selenium import webdriver

# Next 2 lines are needed to specify the path to your geckodriver on
# Ubuntu 2022.04. For other OSes or browsers you may need to fiddle with
# these settings to get a working webdriver.
geckodriver_path = "/snap/bin/geckodriver"
driver_service = webdriver.FirefoxService(executable_path=geckodriver_path)

driver = webdriver.Firefox(service=driver_service)
out = driver.get("https://htc.rescale.com/api-docs/ogrgmNBE+EiA4WoxiZRnHw==/#/")


In [ ]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

# Click all the buttons on each endpoint to expand the sections

buttons = driver.find_elements(By.TAG_NAME, "button")
driver.implicitly_wait(1)

for button in buttons:
    if not "/" in button.text:
        continue

    button.click()
    driver.implicitly_wait(1)

    print("***")
    print(button.text)

    #break

#print(html)



In [ ]:
print("Passing to bs4...")

# Parse the HTML with bs4, grabbing the bits and
# pieces of info we need to create the API JSON spec.

import json
html = driver.page_source
soup = BeautifulSoup(html)

endpoints_patches = soup.find_all(class_="opblock")

res = []

# For each API endpoint
for endpoint_patch in endpoints_patches:
    resitem = {}
    summary = endpoint_patch.find(class_="opblock-summary-path")

    url = summary["data-path"]
    method = endpoint_patch.find(class_="opblock-summary-method").get_text()

    # DEBUG: Only do a single section
    #if url != "/oauth2/token":
    #    continue

    print(f'{method} : {summary["data-path"]}')
    #print(endpoint_patch)

    response_regions = endpoint_patch.find_all(class_="response")
    responses = []
    # For each possible HTTP status code response
    for response_region in response_regions:
        response = {}
        response["status"] = response_region.find(class_="response-col_status").get_text()
        #payload = response_region.find(class_="language-json").get_text()
        print(response["status"])
        payload = response_region.find(class_="highlight-code")
        if payload:
            response["payload"] = payload.get_text()
            print(payload)
        responses.append(response)

    resitem["url"] = url
    resitem["method"] = method
    resitem["responses"] = responses
    res.append(resitem)

with open("api_spec.json", "w") as fp:
    fp.write(json.dumps(res, indent=2))


